In [13]:
!mv ~/home/Documents/Warcraft\ III/CustomMapData/Chlorine/* raw/
!ls -ltr raw/ | tail

mv: cannot stat '/home/b/home/Documents/Warcraft III/CustomMapData/Chlorine/*': No such file or directory
-rwxrwxrwx 1 b b  2647 May  4 23:58 chlorine-stats-Flume-Roost-Scry-960-3.pld
-rwxrwxrwx 1 b b  2647 May  4 23:58 chlorine-stats-Flume-Roost-Scry-960-2.pld
-rwxrwxrwx 1 b b  6046 May  7 21:03 chlorine-stats-Fjord-Yarn-Niche-342-2.pld
-rwxrwxrwx 1 b b 10998 May  7 21:43 chlorine-stats-Urge-Ploy-Joust-545-2.pld
-rwxrwxrwx 1 b b  6871 May  7 22:41 chlorine-stats-Quaint--Quaint-804-2.pld
-rwxrwxrwx 1 b b  4852 May  7 23:49 chlorine-stats-Crisp-Lilt-Ablaze-161-2.pld
-rwxrwxrwx 1 b b  8663 May 14 22:05 chlorine-stats-Opaque-Niche-Pluck-982-2.pld
-rwxrwxrwx 1 b b  4516 May 14 22:44 chlorine-stats-Roost-Mirth-Amble-692-2.pld
-rwxrwxrwx 1 b b  6084 May 23 13:09 chlorine-stats-Bile-Nook-Oust-442-2.pld
-rwxrwxrwx 1 b b 11405 May 23 20:51 chlorine-stats-Arch-Fjord-Flit-826-7.pld


In [2]:
from glob import glob
from pathlib import Path

file_contents = {}

for file in glob("raw/*.pld"):
    pfile = Path(file)
    with open(pfile) as f:
        file_contents[pfile.name] = f.read()

sample = file_contents[[file for file in list(file_contents.keys()) if "Arch-Fjord-" in file][0]]
print(sample[:400] + "...")

function PreloadFiles takes nothing returns nothing

	call PreloadStart()
	call Preload( "")
//v0PT02p2,Overkane#2142PT02p3,FunHD#21324PT02p4,KapaTeJlb#21320PT02p5,ScorpioD1#2338PT02p6,Shymom#1730PT02p7,Cokemonkey11#1442PT02p8,Nvleds#1934PT02p9,billabilla#2820PT16S1p8,Night HowlerPT16S8p8,Start" )
	call Preload( "")
//er CircletPT18S1p3,Tauren GladiatorPT18S8p3,Starter CircletPT23S1p6,Space CadetP...


In [3]:
def clean(file_content: str) -> str:
    valid_lines = [line.strip() for line in file_content.split("\n") if line.startswith("//")]
    cleaned_lines = [line[2:len(line) - 3] for line in valid_lines]
    return "".join(cleaned_lines)

print(clean(sample))

v0PT02p2,Overkane#2142PT02p3,FunHD#21324PT02p4,KapaTeJlb#21320PT02p5,ScorpioD1#2338PT02p6,Shymom#1730PT02p7,Cokemonkey11#1442PT02p8,Nvleds#1934PT02p9,billabilla#2820PT16S1p8,Night HowlerPT16S8p8,Starter CircletPT18S1p3,Tauren GladiatorPT18S8p3,Starter CircletPT23S1p6,Space CadetPT23S8p6,Starter CircletPT28S1p2,Steel ElementalPT28S8p2,Starter CircletPT30S5p0,0,0p1,0,0p2,0,0p3,0,0p4,0,0p5,0,0p6,0,0p7,0,0p8,0,0p9,0,0PT42S1p5,HeadbangerPT42S8p5,Starter CircletPT44S1p7,SorcererPT44S8p7,Starter CircletPT45S1p4,Steel ElementalPT45S8p4,Starter CircletPT52S1p9,Void AnnihilatorPT52S8p9,Starter CircletPT52S8p6,GemPT53S8p7,AmuletPT1M0S6p2,0,0,0,0,p8,0,0,0,0,p6,0,0,0,0,p4,0,0,0,0,p9,0,0,0,0,p3,0,0,0,0,p5,0,0,0,0,p7,0,0,0,0PT1M0S5p0,0,0p1,0,0p2,0,0p3,0,0p4,0,0p5,0,0p6,0,0p7,0,0p8,0,0p9,0,0PT1M30S5p0,42,42p1,56,56p2,0,0p3,0,0p4,26,0p5,0,0p6,0,0p7,284,0p8,49,0p9,0,0PT1M38S8p2,HatPT2M0S6p2,0,0,0,1,p8,0,0,0,0,p6,0,0,0,1,p4,0,0,0,3,p9,0,0,0,1,p3,0,0,0,2,p5,0,0,0,0,p7,0,0,0,1PT2M0S5p0,255,255p1,296,296p2,

In [4]:
from dataclasses import dataclass
import datetime
import isodate
import warnings

SchemaVersion = int

EVENT_TYPE = {
    0: "None",
    1: "ChooseHero",
    2: "Identify",
    3: "Win",
    4: "TowerDeath",
    5: "DamageSnapshot",
    6: "StatSnapshot",
    7: "PlayerLeave",
    8: "GetItem",
}

@dataclass
class Event:
    timestamp: datetime.timedelta
    event_type: int
    player: int
    context: list[str]

def parse_damage_snapshot(chunk: str) -> Event:
    timestamp = chunk.split("S5")[0] + "S"
    context = chunk.split("S5")[1]

    return Event(timestamp=isodate.parse_duration("PT" + timestamp), event_type=5, player=-1, context=[context])

def parse_stat_snapshot(chunk: str) -> Event:
    timestamp = chunk.split("S6")[0] + "S"
    context = chunk.split("S6")[1]

    return Event(timestamp=isodate.parse_duration("PT" + timestamp), event_type=6, player=-1, context=[context])

def parse(code: str) -> tuple[SchemaVersion, list[Event]]:
    chunks = code.split("PT") # TODO: use a streaming parser.

    schema_version = int(chunks[0][1:])

    events = []

    for chunk in chunks[1:]:
        if "S5" in chunk:
            events.append(parse_damage_snapshot(chunk))
            continue

        if "S6" in chunk:
            events.append(parse_stat_snapshot(chunk))
            continue

        timestamp_split = chunk.split("p")

        if not "S" in timestamp_split[0] and timestamp_split[0][0] == "0":
            timestamp_split[0] = timestamp_split[0][0] + "S" + timestamp_split[0][1:]

        if len(timestamp_split) == 1:
            warnings.warn(f"Skipped {chunk} as malformed")
            continue

        timestamp = timestamp_split[0].split("S")[0] + "S"

        event_type = timestamp_split[0].split("S")[1]

        other = "".join(timestamp_split[1:])

        if "S" in timestamp and not timestamp.endswith("S") and len(timestamp.split("S")) == 2:
            timestamp_reproc = timestamp.split("S")
            timestamp = timestamp_reproc[0] + "S"

            other = timestamp_reproc[1] + "," + other

        player_split = other.split(",")

        player = player_split[0]

        context = player_split[1:]

        timestamp_suffix = "" if timestamp.endswith("S") else "S"

        if event_type == '5':
            print(chunk)

        events.append(Event(timestamp=isodate.parse_duration("PT" + timestamp + timestamp_suffix), event_type=int(event_type), player=int(player), context=context))

    return (schema_version, events)

schema_version, events = parse(clean(sample))
print(f"Schema version: {schema_version}")

print(str(events[0]))


Schema version: 0
Event(timestamp=datetime.timedelta(0), event_type=2, player=2, context=['Overkane#2142'])


In [5]:
print([str(event.context) for event in events])

["['Overkane#2142']", "['FunHD#21324']", "['KaaTeJlb#21320']", "['ScorioD1#2338']", "['Shymom#1730']", "['Cokemonkey11#1442']", "['Nvleds#1934']", "['billabilla#2820']", "['Night Howler']", "['Starter Circlet']", "['Tauren Gladiator']", "['Starter Circlet']", "['Sace Cadet']", "['Starter Circlet']", "['Steel Elemental']", "['Starter Circlet']", "['p0,0,0p1,0,0p2,0,0p3,0,0p4,0,0p5,0,0p6,0,0p7,0,0p8,0,0p9,0,0']", "['Headbanger']", "['Starter Circlet']", "['Sorcerer']", "['Starter Circlet']", "['Steel Elemental']", "['Starter Circlet']", "['Void Annihilator']", "['Starter Circlet']", "['Gem']", "['Amulet']", "['p2,0,0,0,0,p8,0,0,0,0,p6,0,0,0,0,p4,0,0,0,0,p9,0,0,0,0,p3,0,0,0,0,p5,0,0,0,0,p7,0,0,0,0']", "['p0,0,0p1,0,0p2,0,0p3,0,0p4,0,0p5,0,0p6,0,0p7,0,0p8,0,0p9,0,0']", "['p0,42,42p1,56,56p2,0,0p3,0,0p4,26,0p5,0,0p6,0,0p7,284,0p8,49,0p9,0,0']", "['Hat']", "['p2,0,0,0,1,p8,0,0,0,0,p6,0,0,0,1,p4,0,0,0,3,p9,0,0,0,1,p3,0,0,0,2,p5,0,0,0,0,p7,0,0,0,1']", "['p0,255,255p1,296,296p2,154,0p3,114,0p4,

In [6]:
import json

with open("cleaned.json", "w") as f:
    f.write(json.dumps([event.__dict__ for event in events], default=str))

In [7]:
# Player identity mapping
# TODO: something is broken for name-processing when name contains "p" i.e. "lep"
player_identity_events = [event for event in events if event.event_type == 2]
print(player_identity_events[0])
player_identities = {identity_event.player: identity_event.context[0].split('#')[0] for identity_event in player_identity_events}
print(list(player_identities.items())[0])

# Player hero choice mapping
hero_choice_events = [event for event in events if event.event_type == 1]
print(hero_choice_events[0])
player_heroes = {hero_event.player: hero_event.context[0] for hero_event in hero_choice_events}
print(list(player_heroes.items())[0])

Event(timestamp=datetime.timedelta(0), event_type=2, player=2, context=['Overkane#2142'])
(2, 'Overkane')
Event(timestamp=datetime.timedelta(seconds=16), event_type=1, player=8, context=['Night Howler'])
(8, 'Night Howler')


In [8]:
# Damage graph
# !pip install matplotlib pandas seaborn
import matplotlib.pyplot as pyplot
import pandas
import seaborn

damage_snapshot_events = [event for event in events if event.event_type == 5]
print(damage_snapshot_events[0].context[0])

def parse_damage_snapshot_context(context: str) -> dict[int, tuple[int, int]]:
    by_player = [cxt for cxt in context.split('p') if len(cxt)]
    ret = {}
    for triple in by_player:
        ret[triple.split(',')[0]] = (triple.split(',')[1], triple.split(',')[2])

    return ret

print(parse_damage_snapshot_context(damage_snapshot_events[0].context[0]))

data ={'time': [], 'player': [], 'damage': [], 'hero_damage': []}

for damage_event_snapshot in damage_snapshot_events:
    for player, (damage, hero_damage) in parse_damage_snapshot_context(damage_event_snapshot.context[0]).items():
        if (int(damage) > 0 or int(hero_damage) > 0) and (int(player) > 1):
            player_name = f"{int(player) + 1} {player_identities.get(int(player))} ({player_heroes.get(int(player))})"
            data['time'].append(damage_event_snapshot.timestamp.total_seconds() / 60.0)
            data['player'].append(player_name)
            data['damage'].append(int(damage))
            data['hero_damage'].append(int(hero_damage))

df = pandas.DataFrame(data)

seaborn.set_theme(style="dark", palette='pastel', context='notebook')

seaborn.lineplot(data=df, x='time', y='damage', hue='player', marker='o')
pyplot.title('Cumulative damage over time by player')
pyplot.xlabel('Time (minutes)')
pyplot.ylabel('Cumulative damage')
pyplot.savefig('damage.png', format='png', dpi=200)
pyplot.close()

# Hero damage
seaborn.lineplot(data=df, x='time', y='hero_damage', hue='player', marker='o')
pyplot.title('Damage dealt to enemy heroes, cumulative over time by player')
pyplot.xlabel('Time (minutes)')
pyplot.ylabel('Cumulative hero damage')
pyplot.savefig('hero_damage.png', format='png', dpi=200)
pyplot.close()

p0,0,0p1,0,0p2,0,0p3,0,0p4,0,0p5,0,0p6,0,0p7,0,0p8,0,0p9,0,0
{'0': ('0', '0'), '1': ('0', '0'), '2': ('0', '0'), '3': ('0', '0'), '4': ('0', '0'), '5': ('0', '0'), '6': ('0', '0'), '7': ('0', '0'), '8': ('0', '0'), '9': ('0', '0')}


In [9]:
# Tower death events
tower_death_events = [event for event in events if event.event_type == 4]
print(tower_death_events[0])

tower_deaths = [[], []]  # Which timestamps did towers die for players 0, 1?

for event in tower_death_events:
    tower_deaths[event.player].append(event.timestamp)

print(tower_deaths)

Event(timestamp=datetime.timedelta(seconds=308), event_type=4, player=0, context=[])
[[datetime.timedelta(seconds=308), datetime.timedelta(seconds=547), datetime.timedelta(seconds=642), datetime.timedelta(seconds=645), datetime.timedelta(seconds=679), datetime.timedelta(seconds=941), datetime.timedelta(seconds=1173), datetime.timedelta(seconds=1197)], [datetime.timedelta(seconds=708), datetime.timedelta(seconds=791), datetime.timedelta(seconds=1030), datetime.timedelta(seconds=1050), datetime.timedelta(seconds=1092), datetime.timedelta(seconds=1241), datetime.timedelta(seconds=1290), datetime.timedelta(seconds=1345)]]


In [10]:
# Resource graph

stat_snapshot_events = [event for event in events if event.event_type == 6]
print(stat_snapshot_events[0].context[0])

def parse_stat_snapshot_context(context: str) -> dict[int, tuple[int, int, int, int]]:
    by_player = [cxt for cxt in context.split('p') if len(cxt)]
    ret = {}
    for triple in by_player:
        ret[triple.split(',')[0]] = (triple.split(',')[1], triple.split(',')[2], triple.split(',')[3], triple.split(',')[4])

    return ret

print(parse_stat_snapshot_context(stat_snapshot_events[0].context[0]))

data ={'time': [], 'player_id': [], 'player': [], 'kills': [], 'assists': [], 'deaths': [], 'cs': [], 'gold_earned': []}

for stat_event_snapshot in stat_snapshot_events:
    for player, (kills, assists, deaths, cs) in parse_stat_snapshot_context(stat_event_snapshot.context[0]).items():
        if (int(kills) > 0 or int(assists) > 0 or int(deaths) > 0 or int(cs) > 0) and (int(player) > 1):
            time = stat_event_snapshot.timestamp
            tower_deaths_so_far = len([death for death in tower_deaths[1 - (int(player) % 2)] if death < time])

            player_name = f"{int(player) + 1} {player_identities.get(int(player))} ({player_heroes.get(int(player))})"
            data['time'].append(stat_event_snapshot.timestamp.total_seconds() / 60.0)
            data['player_id'].append(int(player))
            data['player'].append(player_name)
            data['kills'].append(int(kills))
            data['assists'].append(int(assists))
            data['deaths'].append(int(deaths))
            data['cs'].append(int(cs))
            data['gold_earned'].append(int(kills) * 10 + int(assists) * 5 + int(cs) + 10 * tower_deaths_so_far)

df = pandas.DataFrame(data)

unique_players = df['player_id'].unique()

colors = {
    player_id: seaborn.color_palette("Reds", n_colors=len(unique_players))[i//2]
    if player_id % 2 == 0
    else seaborn.color_palette("Blues", n_colors=len(unique_players))[i//2]
    for i, player_id in enumerate(sorted(unique_players))
}

name_to_color = {name: colors[id] for name, id in zip(df['player'], df['player_id'])}

seaborn.set_theme(style="dark", palette='pastel', context='notebook')

seaborn.lineplot(data=df, x='time', y='gold_earned', hue='player', palette=name_to_color, marker='o')
pyplot.title('Cumulative gold (earned) over time by player')
pyplot.xlabel('Time (minutes)')
pyplot.ylabel('Cumulative gold (earned)')

for time in tower_deaths[0]:
    pyplot.axvline(x=time.total_seconds() / 60.0, color='red', linestyle='--', linewidth=0.5)
    pyplot.text(time.total_seconds() / 60.0, 1, 'West tower death', color='red', horizontalalignment='right', fontsize=4, rotation=15)

for time in tower_deaths[1]:
    pyplot.axvline(x=time.total_seconds() / 60.0, color='blue', linestyle='--', linewidth=0.5)
    pyplot.text(time.total_seconds() / 60.0, 1, 'East tower death', color='blue', horizontalalignment='right', fontsize=4, rotation=15)

victories = [event for event in events if event.event_type == 3]
if len(victories):
    redblue = 'red' if victories[0].player == 0 else 'blue'
    westeast = 'West' if victories[0].player == 0 else 'East'
    pyplot.axvline(x=time.total_seconds() / 60.0, color=redblue, linestyle='--', linewidth=0.5)
    pyplot.text(time.total_seconds() / 60.0, 1, f'{westeast} victory', color='blue', horizontalalignment='right', fontsize=4, rotation=15)

pyplot.savefig('gold.png', format='png', dpi=200)
pyplot.close()

p2,0,0,0,0,p8,0,0,0,0,p6,0,0,0,0,p4,0,0,0,0,p9,0,0,0,0,p3,0,0,0,0,p5,0,0,0,0,p7,0,0,0,0
{'2': ('0', '0', '0', '0'), '8': ('0', '0', '0', '0'), '6': ('0', '0', '0', '0'), '4': ('0', '0', '0', '0'), '9': ('0', '0', '0', '0'), '3': ('0', '0', '0', '0'), '5': ('0', '0', '0', '0'), '7': ('0', '0', '0', '0')}
